---------------------

In [ ]:
!pip install transformers datasets accelerate

In [ ]:
from datasets import load_dataset
dataset = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [ ]:
train_dataset=dataset['train']
validation_dataset=dataset['validation']

In [ ]:
dataset['train']['answers'][0]['text'][0]

'in the late 1990s'

In [ ]:
def read_data(dataset, limit=None):

  contexts = []
  questions = []
  answers = []
  for each_data in dataset:
    if each_data['answers']['text']!=[]:
      contexts.append(each_data['context'])
      questions.append(each_data['question'])
      answers.append(each_data['answers']['text'][0])
                                     
    if limit != None and len(contexts) > limit:
      return contexts, questions, answers           
  return contexts, questions, answers

In [ ]:
train_contexts, train_questions, train_answers = read_data(train_dataset,10000)
eval_contexts, eval_questions, eval_answers = read_data(validation_dataset,1000)

In [ ]:
len(train_answers)

10001

In [ ]:
print(len(train_contexts))
print(len(eval_contexts))

10001
1001


In [ ]:
print('train_contexts: ',train_contexts[0],'\n')
print('train_questions: ',train_questions[0],'\n')
print('train_answers: ',train_answers[0],'\n')

train_contexts:  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". 

train_questions:  When did Beyonce start becoming popular? 

train_answers:  in the late 1990s 



In [ ]:
from transformers import BartTokenizer, AutoConfig
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
config = AutoConfig.from_pretrained('facebook/bart-base')
#train_encodings = tokenizer(train_contexts, train_answers, text_target=train_questions, truncation=True, padding='max_length') #padding='max_length'
train_encodings = tokenizer(train_contexts, train_answers, text_target=train_questions, truncation=True, padding=True)#padding='max_length'A
eval_encodings = tokenizer(eval_contexts, eval_answers, text_target=eval_questions, truncation=True, padding=True)

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
print(train_encodings['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(train_encodings['input_ids'][0]))
print(tokenizer.decode(train_encodings['input_ids'][0]))

print(train_encodings['attention_mask'][0])
print(len(train_encodings['input_ids'][0]))

[0, 40401, 261, 12695, 272, 354, 6591, 10690, 1634, 12, 43732, 48229, 5605, 43621, 16948, 49066, 267, 35423, 10659, 282, 1090, 35423, 10278, 73, 19417, 12, 975, 2191, 12, 28357, 43, 36, 5400, 772, 204, 6, 14130, 43, 16, 41, 470, 3250, 6, 2214, 9408, 6, 638, 3436, 8, 3390, 4, 8912, 8, 1179, 11, 2499, 6, 1184, 6, 79, 3744, 11, 1337, 6970, 8, 7950, 9150, 25, 10, 920, 6, 8, 1458, 7, 9444, 11, 5, 628, 4525, 29, 25, 483, 3250, 9, 248, 947, 387, 1816, 12, 13839, 23313, 18, 7442, 4, 1554, 4628, 30, 69, 1150, 6, 4101, 16152, 10690, 1634, 6, 5, 333, 1059, 65, 9, 5, 232, 18, 275, 12, 11393, 1816, 1134, 9, 70, 86, 4, 2667, 25224, 794, 5, 800, 9, 12674, 12695, 18, 2453, 2642, 6, 34880, 9412, 11, 3437, 36, 35153, 238, 61, 2885, 69, 25, 10, 5540, 3025, 3612, 6, 2208, 292, 12727, 4229, 8, 3520, 5, 18919, 6003, 727, 346, 12, 1264, 7695, 22, 347, 36616, 11, 3437, 113, 8, 22, 30047, 5637, 845, 2, 2, 179, 5, 628, 4525, 29, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
import torch
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')

In [ ]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    get_scheduler
)

In [ ]:
train_batch_size = 8      # 設定 training batch size
eval_batch_size = 8     # 設定 eval batch size
num_train_epochs = 3      # 設定 epoch

In [ ]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, batch_size=eval_batch_size)

In [ ]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 2   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 1878


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
# Train!
output_dir = 'BART_QGmodel_100/'
total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps
print(f"total_batch_size = {total_batch_size}")

completed_steps = 0

for epoch in trange(num_train_epochs, desc="Epoch"):
  model.train()
  train_loss = 0
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    outputs = model(**batch)
    loss = outputs.loss
    train_loss += loss.item()
    loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      completed_steps += 1
      

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})
      
    
    if completed_steps >= max_train_steps:
      break

  train_loss /= len(train_dataloader)
  print(f"epoch : {epoch}, train_loss: {train_loss}")

  model.eval()
  eval_loss = 0
  for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):
    outputs = model(**batch)
    eval_loss += outputs.loss.item()

  eval_loss /= len(eval_dataloader)
  print(f"epoch : {epoch}, eval_loss: {eval_loss}")

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(f"{output_dir}epoch_{epoch}_train_loss={train_loss}_eval_loss={eval_loss}/", save_function=accelerator.save)
  

total_batch_size = 16


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1251 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 6.226309299468994}
{'epoch': 0, 'step': 50, 'loss': 2.484238624572754}
{'epoch': 0, 'step': 100, 'loss': 1.4922804832458496}
{'epoch': 0, 'step': 150, 'loss': 0.8889921307563782}
{'epoch': 0, 'step': 200, 'loss': 0.6312387585639954}
{'epoch': 0, 'step': 250, 'loss': 0.4478236436843872}
{'epoch': 0, 'step': 300, 'loss': 0.378597617149353}
{'epoch': 0, 'step': 350, 'loss': 0.3743389844894409}
{'epoch': 0, 'step': 400, 'loss': 0.3876713514328003}
{'epoch': 0, 'step': 450, 'loss': 0.40921247005462646}
{'epoch': 0, 'step': 500, 'loss': 0.30348631739616394}
{'epoch': 0, 'step': 550, 'loss': 0.36020341515541077}
{'epoch': 0, 'step': 600, 'loss': 0.28589117527008057}
{'epoch': 0, 'step': 650, 'loss': 0.20154808461666107}
{'epoch': 0, 'step': 700, 'loss': 0.33078181743621826}
{'epoch': 0, 'step': 750, 'loss': 0.3117985427379608}
{'epoch': 0, 'step': 800, 'loss': 0.2688950300216675}
{'epoch': 0, 'step': 850, 'loss': 0.3486459255218506}
{'epoch': 0, 'step': 900, 'l

Eval Iteration:   0%|          | 0/126 [00:00<?, ?it/s]

...

epoch : 1, eval_loss: 0.6492930887237428


Iteration:   0%|          | 0/1251 [00:00<?, ?it/s]

{'epoch': 2, 'step': 0, 'loss': 0.25786054134368896}
{'epoch': 2, 'step': 50, 'loss': 0.2800181210041046}
{'epoch': 2, 'step': 100, 'loss': 0.2182697355747223}
{'epoch': 2, 'step': 150, 'loss': 0.2463739961385727}
{'epoch': 2, 'step': 200, 'loss': 0.28401848673820496}
{'epoch': 2, 'step': 250, 'loss': 0.22863253951072693}
{'epoch': 2, 'step': 300, 'loss': 0.1853473037481308}
{'epoch': 2, 'step': 350, 'loss': 0.18790148198604584}
{'epoch': 2, 'step': 400, 'loss': 0.20604254305362701}
{'epoch': 2, 'step': 450, 'loss': 0.18821877241134644}
{'epoch': 2, 'step': 500, 'loss': 0.33468982577323914}
{'epoch': 2, 'step': 550, 'loss': 0.17295345664024353}
{'epoch': 2, 'step': 600, 'loss': 0.25429609417915344}
{'epoch': 2, 'step': 650, 'loss': 0.20745548605918884}
{'epoch': 2, 'step': 700, 'loss': 0.14953893423080444}
{'epoch': 2, 'step': 750, 'loss': 0.2057746797800064}
{'epoch': 2, 'step': 800, 'loss': 0.2597790062427521}
{'epoch': 2, 'step': 850, 'loss': 0.26958781480789185}
{'epoch': 2, 'step'

Eval Iteration:   0%|          | 0/126 [00:00<?, ?it/s]

epoch : 2, eval_loss: 0.6484358123843632


In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
path = "/content/BART_QGmodel_100/epoch_2_train_loss=0.4736572372779953_eval_loss=0.6484358123843632"
model = AutoModelForSeq2SeqLM.from_pretrained(path)

In [ ]:
context = '''Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic and subjugate all wizards and Muggles (non-magical people).'''
#question = 'Who is the author of Harry Potter?'
answer = 'J. K. Rowling'
input = tokenizer([context], [answer],truncation=True, padding=True,return_tensors="pt")

In [ ]:
bs = 5
outputs = model.generate(
                        **input,
                        early_stopping=True,     #提早停止
                        num_beams=bs+3,          #往後延伸多少level的分支
                        no_repeat_ngram_size=5,  #5: Harry Potter is a series , 2: Harry Potter
                        num_return_sequences=bs, #回傳的句子數量
                )
for output in outputs:
  print(tokenizer.decode(output, skip_special_tokens=True))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Who wrote Harry Potter?
Who wrote the book Harry Potter?
Who wrote the Harry Potter series?
Who wrote the seven fantasy novels?
Who created Harry Potter?


In [ ]:
context = 'Sun cake is a kind of pancake with sweet filling, usually filled with maltose. It originated from the malt cake made by Lin Jiakun in Shekou, Shengang District, Taichung City. It is a snack in Taichung City, Taiwan, and one of the famous products in Central Taiwan. The name Sun Cake has not been registered as a trademark, so that later stores in the same industry can use this name. The shape of sun cakes is approximately round. In the early days, the sun cakes were made relatively large. Generally, they were divided into four pieces before eating. In recent years, smaller sun cakes have appeared for easy eating.'
#question = 'What is the filling of Sun cake'
answer = 'maltose'
input = tokenizer([context], [answer],truncation=True, padding=True,return_tensors="pt")

In [ ]:
bs = 5
outputs = model.generate(
                        **input,
                        early_stopping=True,     #提早停止
                        num_beams=bs+3,          #往後延伸多少level的分支
                        no_repeat_ngram_size=5,  #5: Harry Potter is a series , 2: Harry Potter
                        num_return_sequences=bs, #回傳的句子數量
                )
for output in outputs:
  print(tokenizer.decode(output, skip_special_tokens=True))

What is the filling of Sun cake?
What is the sweet filling of Sun cake?
What is the filling of Sun Cake?
What is the sweet filling of Sun Cake?
What is the filling of a Sun cake?
